In [6]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [7]:
# Define paths and file names 
target_path = 'Targets/daily_crsp.csv'

firm_charac_path = 'Predictors/CompFirmCharac.csv'

link_table_path = 'linking_table.csv'

# First date for which we have daily returns
CUTOFF_DATE = '2000-01-03'

In [15]:
def import_sanitize_daily_returns(target_path, nrows=None):
    """
    Import and sanitize the data frame of daily returns.

    Parameters:
     - target_path: the path to the CSV file containing the daily returns 
     - nrows: number of rows to load from the file. None if the whole dataset is to be loaded.
    """

    required_columns = ['PERMNO', 'PERMCO', 'DlyCalDt', 'SICCD', 'NAICS', 'DlyRet', 'sprtrn']
    daily_data = pd.read_csv(target_path, usecols=required_columns, nrows=nrows)

    nan_counts = daily_data[['PERMNO', 'DlyCalDt', 'DlyRet']].isna().sum()
    total_observations = len(daily_data)
    total_nans = daily_data['DlyRet'].isna().sum()
    overall_nan_pct = (total_nans / total_observations) * 100

    print("\n===== NaN Pattern Breakdown =====")
    nan_mask = daily_data[['PERMNO', 'DlyCalDt', 'DlyRet']].isna()
    print(f"PERMNO only     : {(nan_mask['PERMNO'] & ~nan_mask['DlyCalDt'] & ~nan_mask['DlyRet']).sum():,}")
    print(f"DlyCalDt only   : {(~nan_mask['PERMNO'] & nan_mask['DlyCalDt'] & ~nan_mask['DlyRet']).sum():,}")
    print(f"DlyRet only     : {(~nan_mask['PERMNO'] & ~nan_mask['DlyCalDt'] & nan_mask['DlyRet']).sum():,}")
    print(f"Total NaN Percentage: {overall_nan_pct:.2f}%")
 

    # PERMNO-level missing summary
    permno_stats = daily_data.groupby('PERMNO')['DlyRet'].agg(['size', lambda x: x.isna().sum()])
    permno_stats.columns = ['Total_Obs', 'NaN_Count']
    permno_stats['NaN_%'] = (permno_stats['NaN_Count'] / permno_stats['Total_Obs']) * 100

    permnos_to_remove = permno_stats[permno_stats['NaN_%'] > 30].index
    print(f"\nPERMNOs with >30% missing : {len(permnos_to_remove):,}")
    print(f"Total PERMNOs before drop : {daily_data['PERMNO'].nunique():,}")

    # Filter out bad PERMNOs
    daily_data = daily_data[~daily_data['PERMNO'].isin(permnos_to_remove)] 

    
    # Detect outliers (extreme returns)
    lower_q = daily_data['DlyRet'].quantile(0.00005)
    upper_q = daily_data['DlyRet'].quantile(0.99995)
    outliers = (daily_data['DlyRet'] < lower_q) | (daily_data['DlyRet'] > upper_q)

    print("\n===== Outlier Detection =====")
    print(f"Lower quantile (0.005%)   : {lower_q:.4f}")
    print(f"Upper quantile (99.995%)  : {upper_q:.4f}")
    print(f"Outliers flagged          : {outliers.sum():,} rows")

        # Winsorize (cap outliers)
    daily_data['DlyRet'] = daily_data['DlyRet'].clip(lower=lower_q, upper=upper_q)


    # Impute remaining missing with PERMNO-level mean
    daily_data['DlyRet'] = daily_data.groupby('PERMNO')['DlyRet'].transform(lambda x: x.fillna(x.mean()))

    # Date handling & final formatting
    daily_data['DlyCalDt'] = pd.to_datetime(daily_data['DlyCalDt'], errors='coerce')
    daily_data = daily_data.rename(columns={'DlyCalDt': 'date'})
    daily_data = daily_data.sort_values(by=['date', 'PERMCO', 'PERMNO'])
    daily_data = daily_data.drop_duplicates(subset=['PERMNO', 'date'])
    daily_data = daily_data.set_index(['date', 'PERMCO', 'PERMNO'])

    print("\n===== Final Dataset Summary =====")
    print(f"Remaining observations    : {len(daily_data):,}")
    print(f"Final PERMNO count        : {daily_data.index.get_level_values('PERMNO').nunique():,}")
    print(f"Date range                : {daily_data.index.get_level_values('date').min().date()} to {daily_data.index.get_level_values('date').max().date()}")

    return daily_data

In [16]:
daily_data = import_sanitize_daily_returns(target_path, nrows=None)

MemoryError: Unable to allocate 512. KiB for an array with shape (65536,) and data type int64

In [5]:
display(daily_data)

SICCD   NAICS    DlyRet    sprtrn
date       PERMCO PERMNO                                   
2000-01-03 5      15580    6320       0  0.000000 -0.009549
           7      14593    3573       0  0.088754 -0.009549
           25     62770    6711       0 -0.061489 -0.009549
           29     59184    2082       0 -0.012346 -0.009549
           33     59248    2082       0 -0.034524 -0.009549
...                         ...     ...       ...       ...
2024-12-31 60118  26023    9999       0 -0.000393 -0.004285
           60120  26027    9999  523910 -0.194924 -0.004285
           60121  26034    9999       0  0.108590 -0.004285
           60122  26035    9999  551112 -0.034979 -0.004285
           60123  26037       0  423610  0.059333 -0.004285

[46468773 rows x 4 columns]

In [6]:
daily_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 46468773 entries, (Timestamp('2000-01-03 00:00:00'), np.int64(5), np.int64(15580)) to (Timestamp('2024-12-31 00:00:00'), np.int64(60123), np.int64(26037))
Data columns (total 4 columns):
 #   Column  Dtype  
---  ------  -----  
 0   SICCD   int64  
 1   NAICS   int64  
 2   DlyRet  float64
 3   sprtrn  float64
dtypes: float64(2), int64(2)
memory usage: 1.6 GB


In [7]:
daily_data.describe()

,SICCD,NAICS,DlyRet,sprtrn
count,4.646877e+07,4.646877e+07,4.646877e+07,4.646877e+07
mean,5.784461e+03,3.722121e+05,4.534362e-04,2.927101e-04
std,2.215948e+03,2.070388e+05,3.895725e-02,1.218518e-02
min,0.000000e+00,0.000000e+00,-5.812940e-01,-1.198410e-01
25%,3.820000e+03,3.118120e+05,-1.079600e-02,-4.849000e-03
50%,6.410000e+03,5.112100e+05,0.000000e+00,5.890000e-04
75%,6.726000e+03,5.259900e+05,1.043200e-02,6.002000e-03
max,9.999000e+03,9.999900e+05,1.167635e+00,1.158000e-01


In [ ]:
import os

def save_sanitized_csv(df, original_path, date_format='%Y-%m-%d'):
    """
    Save the sanitized DataFrame to a new CSV with '_sanitized' appended to the original filename.

    Parameters:
     - df: The sanitized DataFrame (with a datetime index).
     - original_path: The original CSV file path.
     - date_format: Format for datetime values in the output file.
    """
    # Get directory, filename, and extension
    base, ext = os.path.splitext(original_path)
    new_path = f"{base}_sanitized{ext}"

    # Reset index to save multi-index as columns
    df_to_save = df.reset_index()

    # Write to CSV with datetime formatting
    df_to_save.to_csv(new_path, index=False, date_format=date_format)

    print(f"Sanitized data saved to: {new_path}")

In [9]:
save_sanitized_csv(daily_data, target_path)

Sanitized data saved to: Targets/daily_crsp_sanitized.csv


In [ ]:
def import_sanitize_firm_charac(
    firm_charac_path, 
    nrows=None, 
    missing_threshold=0.3, 
    required_unique=500, 
    cutoff_date='2000-01-03'
):
    """
    Imports and sanitizes the dataset containing firm characteristics.

    Parameters:
     - firm_charac_path: path to CSV file containing the data
     - nrows: number of rows to load in memory (None if the whole dataset is to be loaded)
     - missing_threshold: if a column contains more than missing_threshold*100% missing values, it is dropped
     - required_unique: if a column has fewer than required_unique unique values, it is dropped
     - cutoff_date: data starts from this date (inclusive)
    """
    # Load data and convert date column
    comp = pd.read_csv(firm_charac_path, nrows=nrows, low_memory=False)
    comp['datadate'] = pd.to_datetime(comp['datadate'], errors='coerce')

    # Apply start and end date filters
    comp = comp[comp['datadate'] >= pd.to_datetime(cutoff_date)]
    comp = comp[comp['datadate'] <= pd.to_datetime('2024-12-31')]

    # Drop missing gvkey or datadate and remove duplicates
    comp = comp.dropna(subset=['gvkey', 'datadate'])
    comp = comp.drop_duplicates(subset=['gvkey', 'datadate'])

    # Rename and reindex
    comp = comp.rename(columns={'datadate': 'date'})
    comp = comp.sort_values(by=['date', 'gvkey'])
    comp = comp.set_index(['date', 'gvkey'])

    # Drop identifier columns
    identifiers = ['cusip', 'tic', 'conm', 'exchg', 'cik', 'costat', 'fic']
    comp = comp.drop(columns=[col for col in identifiers if col in comp.columns])

    # Drop columns with too many missing values
    comp = comp.loc[:, comp.isna().mean() < missing_threshold]

    # Drop columns with too few unique values
    comp = comp.loc[:, comp.nunique(dropna=True) >= required_unique]

    # Calculate missing value share per firm and retain those below threshold
    firm_missing_pct = comp.groupby(level='gvkey').apply(
        lambda df: df.isna().sum().sum() / (len(df) * len(df.columns))
    )
    firms_to_keep = firm_missing_pct[firm_missing_pct <= missing_threshold].index

    print("\n===== Firm Filtering =====")
    print(f"Firms with ≤{missing_threshold*100:.0f}% missing data: {len(firms_to_keep):,}")

    # Filter dataset to keep only selected firms
    comp_filtered = comp[comp.index.get_level_values('gvkey').isin(firms_to_keep)]

    # Winsorization (cap extreme outliers)
    numeric_cols = comp_filtered.select_dtypes(include='number').columns
    total_outliers = 0

    for col in numeric_cols:
        lower_q = comp_filtered[col].quantile(0.00005)
        upper_q = comp_filtered[col].quantile(0.99995)
        outliers = (comp_filtered[col] < lower_q) | (comp_filtered[col] > upper_q)
        total_outliers += outliers.sum()
        comp_filtered[col] = comp_filtered[col].clip(lower=lower_q, upper=upper_q)

    print("\n===== Winsorization Summary =====")
    avg_outliers_per_col = total_outliers / len(numeric_cols)
    print(f"Avg. outliers capped per column: {avg_outliers_per_col:,.1f}")

    # Track missing values before imputation
    missing_before = comp_filtered.isna().sum()

    # Impute missing values using forward-fill, backward-fill, and then median by date
    comp_filtered = comp_filtered.groupby(level='gvkey').ffill()
    comp_filtered = comp_filtered.groupby(level='gvkey').bfill()
    comp_filtered = comp_filtered.groupby(level='date').apply(
        lambda group: group.fillna(group.median(numeric_only=True))
    ).droplevel(0)

    # Diagnostics on imputation
    missing_after = comp_filtered.isna().sum()
    reduction = missing_before - missing_after
    reduction_pct = ((reduction / missing_before) * 100).fillna(0)

    # Final diagnostics
    print(f"\n===== Final Overview =====")
    print(f"Remaining columns     : {len(comp_filtered.columns):,}")
    print(f"Column names          : {comp_filtered.columns.tolist()}")
    print(f"Final firm count      : {comp_filtered.index.get_level_values('gvkey').nunique():,}")
    print(f"Date range            : {comp_filtered.index.get_level_values('date').min().date()} → {comp_filtered.index.get_level_values('date').max().date()}")

    return comp_filtered


In [15]:
firms = import_sanitize_firm_charac(firm_charac_path, cutoff_date=CUTOFF_DATE)



===== Firm Filtering =====
Firms with ≤30% missing data: 21,280

===== Winsorization Summary =====
Avg. outliers capped per column: 81.1

===== Final Overview =====
Remaining columns     : 41
Column names          : ['acchgy', 'capxy', 'chechy', 'cogsy', 'cshfdy', 'cshpry', 'cstkey', 'dilady', 'dilavy', 'doy', 'dvpy', 'dvy', 'epsfiy', 'epsfxy', 'epspiy', 'epspxy', 'exrey', 'fiaoy', 'fincfy', 'ibadjy', 'ibcomy', 'iby', 'ivacoy', 'ivncfy', 'miiy', 'niy', 'nopiy', 'oancfy', 'oepsxy', 'oiadpy', 'oibdpy', 'opepsy', 'piy', 'revty', 'saley', 'spiy', 'sstky', 'txty', 'xidoy', 'xiy', 'xopry']
Final firm count      : 21,280
Date range            : 2000-01-31 → 2024-12-31


In [16]:
firms.describe()

,acchgy,capxy,chechy,cogsy,cshfdy,cshpry,cstkey,dilady,dilavy,doy,...,opepsy,piy,revty,saley,spiy,sstky,txty,xidoy,xiy,xopry
count,871758.000000,871758.000000,871758.000000,871758.000000,871758.000000,871758.000000,871758.000000,871758.000000,871758.000000,871758.000000,...,871758.000000,871758.000000,871758.000000,871758.000000,871758.000000,871758.000000,871758.000000,871758.000000,871758.000000,871758.000000
mean,-0.651047,125.457289,18.457409,1165.884095,177.004368,163.261417,-0.306790,0.162563,110.755352,2.200917,...,-6.739159,167.133944,1663.633790,1729.790463,-16.290347,36.360440,47.047998,1.422752,-0.603659,1401.248060
std,24.829515,833.319270,1157.428647,7445.605687,3554.733450,2640.296160,9.605914,8.056603,1019.896930,106.928299,...,918.447597,1356.785842,9824.247799,9951.407154,284.589664,414.871862,392.975225,117.081076,26.761533,8413.944720
min,-2055.081860,-13.874545,-52326.959500,-920.692430,0.000000,0.000000,-686.136000,-498.000000,-21113.267500,-4048.045150,...,-83614.020500,-21699.340000,-1545.216042,-1516.144505,-16399.863650,-111.428558,-6220.252400,-5216.772800,-2205.400000,-729.767474
25%,0.000000,0.104000,-5.189000,2.983000,14.818000,14.500000,0.000000,0.000000,-3.402000,0.000000,...,-0.090000,-3.392000,5.151000,4.839000,-0.647000,0.000000,0.000000,0.000000,0.000000,7.955000
50%,0.000000,1.986000,0.000000,32.510000,36.810000,36.190000,0.000000,0.000000,0.398000,0.000000,...,0.040000,0.828000,70.047500,62.868000,0.000000,0.141000,0.136000,0.000000,0.000000,54.542000
75%,0.000000,21.668000,6.870000,287.578000,92.980000,91.568000,0.000000,0.000000,20.593000,0.000000,...,0.640000,33.621000,458.287500,495.563750,0.000000,4.032000,7.621000,0.000000,0.000000,393.453000
max,373.419100,35091.706450,72929.726000,317601.081153,432561.609677,339741.632000,323.802896,481.857950,54039.771000,8419.899750,...,26763.141270,66783.605000,420172.514885,409921.592261,10127.502026,25819.627812,19680.692858,8378.907000,717.090000,359547.837200


In [17]:
display(firms)

acchgy     capxy   chechy      cogsy    cshfdy    cshpry  \
date       gvkey                                                              
2000-01-31 1013       0.0    46.785   21.138    291.300   182.525   174.925   
           1082       0.0     9.597   -2.956    117.073     2.936     2.936   
           1173       0.0     0.558   -0.754     14.390     3.937     3.937   
           1183       0.0     1.091   22.224      9.732    28.169    28.169   
           1189       0.0    17.181  -12.630    724.161    31.445    30.435   
...                   ...       ...      ...        ...       ...       ...   
2024-12-31 351491     0.0   972.159  843.780  12427.891   271.000   267.000   
           351590     0.0  1467.038 -532.151  43200.489  1592.000  1592.000   
           352262     0.0   187.490   28.382     77.330    53.701    53.701   
           356128     0.0   182.467 -383.127    635.416   191.430   189.832   
           356289     0.0     0.000    8.492     14.242    30.725    30.725   

                   cstkey  dilady    dilavy      doy  ...  opepsy       piy  \
date       gvkey                                      ...                     
2000-01-31 1013       0.0     0.0    53.100    0.000  ...    0.31    68.400   
           1082       0.0     0.0     0.913   -2.563  ...    0.31     1.503   
           1173       0.0     0.0     0.260    0.000  ...    0.07     0.482   
           1183       0.0     0.0   -14.056    0.741  ...   -0.13   -17.342   
           1189       0.0     0.0    17.295   -0.007  ...    0.57    31.030   
...                   ...     ...       ...      ...  ...     ...       ...   
2024-12-31 351491     0.0     0.0   553.892 -192.568  ...    2.17   629.470   
           351590     0.0     0.0  3002.407    0.000  ...    2.40  3936.259   
           352262     0.0     0.0    98.142    0.000  ...    1.83   101.077   
           356128     0.0     0.0  1981.888    0.000  ...   10.44  2444.223   
           356289     0.0     0.0    -0.070    0.000  ...    0.00    -1.216   

                       revty      saley     spiy   sstky     txty    xidoy  \
date       gvkey                                                             
2000-01-31 1013      593.900    593.900    0.000  42.657   15.300    0.000   
           1082      131.177    131.177    0.000   0.000    0.590   -2.563   
           1173       23.271     23.271    0.000   0.000    0.222    0.000   
           1183        7.476      7.476  -15.974   0.000   -3.286    0.741   
           1189      779.820    779.820    0.000   0.000   12.136   -0.007   
...                      ...        ...      ...     ...      ...      ...   
2024-12-31 351491  15828.894  15828.894 -150.121   0.000   72.472 -192.568   
           351590  55986.599  55986.599 -499.021   0.000  761.991    0.000   
           352262    338.497    338.497    0.000   0.000    0.277    0.000   
           356128   4826.644   4826.644    0.000   0.000  429.749    0.000   
           356289     20.102     20.102    0.000   0.395    0.641    0.000   

                   xiy      xopry  
date       gvkey                   
2000-01-31 1013    0.0    496.900  
           1082    0.0    123.541  
           1173    0.0     21.847  
           1183    0.0      9.732  
           1189    0.0    724.161  
...                ...        ...  
2024-12-31 351491  0.0  13815.210  
           351590  0.0  50543.961  
           352262  0.0     99.266  
           356128  0.0   1153.056  
           356289  0.0     20.722  

[871758 rows x 41 columns]

In [18]:
firms.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 871758 entries, (Timestamp('2000-01-31 00:00:00'), np.int64(1013)) to (Timestamp('2024-12-31 00:00:00'), np.int64(356289))
Data columns (total 41 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   acchgy  871758 non-null  float64
 1   capxy   871758 non-null  float64
 2   chechy  871758 non-null  float64
 3   cogsy   871758 non-null  float64
 4   cshfdy  871758 non-null  float64
 5   cshpry  871758 non-null  float64
 6   cstkey  871758 non-null  float64
 7   dilady  871758 non-null  float64
 8   dilavy  871758 non-null  float64
 9   doy     871758 non-null  float64
 10  dvpy    871758 non-null  float64
 11  dvy     871758 non-null  float64
 12  epsfiy  871758 non-null  float64
 13  epsfxy  871758 non-null  float64
 14  epspiy  871758 non-null  float64
 15  epspxy  871758 non-null  float64
 16  exrey   871758 non-null  float64
 17  fiaoy   871758 non-null  float64
 18  fincfy  871758 non-null  floa

In [19]:
save_sanitized_csv(firms, firm_charac_path)

Sanitized data saved to: Predictors/CompFirmCharac_sanitized.csv


In [ ]:
def import_sanitize_linking_table(link_table_path):
    ccm = pd.read_csv(link_table_path)

    # Convert start date
    ccm['LINKDT'] = pd.to_datetime(ccm['LINKDT'])

    # Replace 'E' with a placeholder date, then convert and fill missing values
    ccm['LINKENDDT'] = ccm['LINKENDDT'].replace('E', '2099-12-31')
    ccm['LINKENDDT'] = pd.to_datetime(ccm['LINKENDDT'], errors='coerce')
    ccm['LINKENDDT'] = ccm['LINKENDDT'].fillna(pd.to_datetime('2099-12-31'))

    # Rename columns for merge compatibility
    ccm.rename(columns={'GVKEY': 'gvkey', 'LPERMNO': 'PERMNO', 'LPERMCO': 'PERMCO'}, inplace=True)

    ccm = ccm.set_index('gvkey')

    return ccm

In [21]:
links = import_sanitize_linking_table(link_table_path)

In [22]:
links

,LINKTYPE,PERMNO,PERMCO,LINKDT,LINKENDDT
gvkey,,,,,
1000,LU,25881,23369,1970-11-13,1978-06-30
1001,LU,10015,6398,1983-09-20,1986-07-31
1002,LC,10023,22159,1972-12-14,1973-06-05
1003,LU,10031,6672,1983-12-07,1989-08-16
1004,LU,54594,20000,1972-04-24,2099-12-31
...,...,...,...,...,...
352262,LC,23773,59507,2023-03-17,2099-12-31
353444,LC,23209,59330,2022-07-22,2099-12-31
355398,LC,25134,59886,2024-05-17,2099-12-31


In [23]:
save_sanitized_csv(links, link_table_path)

Sanitized data saved to: linking_table_sanitized.csv
